In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go

In [ ]:


# Define the matrices
X = np.array([[0, 1, 1, 2, 2],
              [3, 3, 4, 4, 5],
              [5, 6, 6, 7, 7]])

Y = np.array([[0, 1, 1, 2],
              [2, 3, 3, 4],
              [4, 5, 5, 6]])

Z = np.array([[ 2,  5,  7, 10],
              [ 2,  4,  7,  9],
              [ 1,  4,  6,  9],
              [ 1,  3,  6,  8],
              [ 0,  3,  5,  8]])

# Put them into a dictionary
a = {'X': X, 'Y': Y, 'Z': Z}

a

In [ ]:
# this function is for the dict type matrix: a
def v3_cal(a):
    # get the size of the matrix a: n_x,n_y,n_z
    n_x = a[list(a)[2]].shape[1]
    n_y = a[list(a)[2]].shape[0]
    n_z = a[list(a)[0]].shape[0]
    volumes = np.zeros((n_x, n_y, n_z))
    a_z = np.rot90(a[list(a)[2]], k=-1) # k= -1 顺时针旋转90度。
    for i in range(n_x):
        for j in range(n_z):
            for k in range(n_y):
                volumes[i, k, j] = (a[list(a)[0]][j, k] + a[list(a)[1]][j, i] + a_z[i, k])/3
    avg = volumes.mean()
    v1 = np.flip(volumes)
    v2 = np.fliplr(v1)
    v3 = np.flipud(v2)
    return v3,avg

In [ ]:
v3,avg = v3_cal(a)
np.shape(v3)

In [ ]:
def plot2D_simple(X):
    ax = plt.gca()
    im1 = ax.imshow(X, aspect='equal',cmap='plasma_r', interpolation='nearest')
    # for i in range(X.shape[0]):
    #     for j in range(X.shape[1]):
    #         plt.text(j, i, f"{X[i, j]:.2f}", ha='center', va='center', color='k')
            
def plot2D_3_simple(a):
    fig, axs = plt.subplots(1, 3, figsize=(18, 8),dpi=300)
    plt.sca(axs[0])
    plot2D_simple(a[list(a)[0]])
    plt.title("X")
    plt.sca(axs[1])
    plot2D_simple(a[list(a)[1]])
    plt.title("Y")
    plt.sca(axs[2])
    plot2D_simple(a[list(a)[2]])
    plt.title("Z")

def plot2D(X):
    ax = plt.gca()
    im1 = ax.imshow(X, aspect='equal',cmap='plasma_r', interpolation='nearest')

def plot2D_3(a):
    fig, axs = plt.subplots(1, 3, figsize=(18, 8),dpi=300)
    plt.sca(axs[0])
    plot2D(a[0])
    plt.title("X")
    plt.sca(axs[1])
    plot2D(a[1])
    plt.title("Y")
    plt.sca(axs[2])
    plot2D(a[2])
    plt.title("Z")
plot2D_3_simple(a)

In [ ]:
def volume3D(matrix,data_name,sample_name,colorbar_unit,df_avg):
    x, y, z = np.indices(matrix.shape)
    fig = go.Figure(data=go.Isosurface(
        x=x.flatten(),
        y=y.flatten(),
        z=z.flatten(),
        value=matrix.flatten(),
        isomin=matrix.min(),
        isomax=matrix.max(),
        colorscale='plasma_r',
        colorbar=dict(
            title=colorbar_unit,
            x=1,  # Set the x position of the color bar
            y=0.5,  # Set the y position of the color bar
            len=0.75,  # Set the length of the color bar
            thickness=20,  # Set the thickness of the color bar
            orientation='v'  # Set the orientation of the color bar
        ),
    ))
    fig.update_layout(
        title= sample_name+" "+data_name+"_avg = "+str(round(df_avg,2))+colorbar_unit,
        width=800,
        height=800,
        scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z')
    )
            
        
    name = 'eye = (x:2, y:2, z:2)'
    camera = dict(eye=dict(x=1.2,y=-2,z=0.8))
    fig.update_layout(scene_camera=camera)
    # pio.write_image(fig, "Results/"+sample_name+"_"+data_name+'_3D.png',scale = 3)
    fig.show()

In [ ]:
volume3D(v3,"data","sample","unit",avg)

#  Create a pseudo 3x3x3 matrix and interpolate it to be Lx, Ly, Lz

In [ ]:
import os, sys
ROOT_DIR = os.path.abspath(os.getcwd())
sys.path.append(ROOT_DIR)  
from coalpro import utils
from coalpro import visualize

In [ ]:
def volume_data_xyz(E_2D,Lx,Ly,Lz,method):
    E = []
    for selected_face in ['X','Y','Z']:
        grid_x, grid_y, grid_2d = utils.matrix_interpo_xyz(E_2D, Lx, Ly, Lz, selected_face ,method)
        E.append(grid_2d)
        
    n_x = E[2].shape[1]
    n_y = E[2].shape[0]
    n_z = E[0].shape[0]
    volumes = np.zeros((n_x, n_y, n_z))
    
    E_z = np.rot90(E[2], k=-1) # k= -1 顺时针旋转90度。
    for i in range(n_x):
        for j in range(n_z):
            for k in range(n_y):
                volumes[i, k, j] = (E[0][j, k] + E[1][j, i] + E_z[i, k])/3
    avg = round(volumes.mean(),2)
    print("Average Volume:", avg)
    E1 = np.flip(volumes)
    E2 = np.fliplr(E1)
    E3 = np.flipud(E2)
    return E,E3, avg

In [ ]:
# Define the matrices
Ex = np.array([[0, 1, 2],
              [3, 4, 5],
              [6, 7, 8]])
Ey = Ex
Ez = np.array([[ 2,  5,  8],
              [ 1,  4,  7],
              [ 0,  3,  6]])
# Put them into a dictionary
E2D = np.stack((Ex, Ey, Ez), axis=0)
E2D
# Put them into a dictionary
E2D_dict= {'X': Ex, 'Y': Ey, 'Z': Ez}

E2D_dict

In [ ]:
E2D

In [ ]:
Ev3,Eavg = v3_cal(E2D_dict)
# every sub cube multiple each other

In [ ]:
Lx = 40
Ly = 50
Lz = 30
method ='rbf'
E,E3, avg = volume_data_xyz(E2D,Lx,Ly,Lz,method)

In [ ]:
plot2D_3_simple(E2D_dict)
plot2D_3(E)


In [ ]:
volume3D(Ev3,"data","sample","unit",Eavg)
volume3D(E3,"data","sample","unit",avg)